In [1]:
# final code without perfect decryption
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from PIL import Image, ImageTk
import os


def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png *.jpg *.jpeg")])
    if file_path:
        image = Image.open(file_path)
        image = image.resize((400, 400))  # Resize for display purposes
        image_tk = ImageTk.PhotoImage(image)
        input_image_label.config(image=image_tk)
        input_image_label.image = image_tk  # Keep a reference to avoid garbage collection
        encrypt_button.config(state=tk.NORMAL)


# Function to encrypt the image
def encrypt_image():
    # Open file dialog to select input image
    input_file_path = filedialog.askopenfilename(title="Select Input Image")
    if not input_file_path:
        return

    # Retrieve the encryption key from the entry field
    encryption_key = encryption_key_entry.get()

    # Load original image
    imageOrig = cv2.imread(input_file_path)
    rowOrig, columnOrig, depthOrig = imageOrig.shape

    # Convert original image data to bytes
    imageOrigBytes = imageOrig.tobytes()

    # Generate random salt
    salt = os.urandom(16)

    # Generate key from password using PBKDF2
    password = encryption_key.encode()  # Convert the key to bytes
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
    )
    key = kdf.derive(password)

    # Generate random nonce
    nonce = os.urandom(12)

    # Encrypt
    cipher = Cipher(algorithms.AES(key), modes.GCM(nonce), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(imageOrigBytes) + encryptor.finalize()

    # Generate authentication tag
    tag = encryptor.tag[:16]  # Limit the tag length to 16 bytes

    # Convert ciphertext bytes to encrypted image data
    imageEncrypted = np.frombuffer(ciphertext, dtype=np.uint8).reshape(rowOrig, columnOrig, depthOrig)

    # Display encrypted image
    cv2.imshow("Encrypted image", imageEncrypted)
    cv2.waitKey()
    print(tag)

    # Save the encrypted image
    output_file_path = filedialog.asksaveasfilename(title="Save Encrypted Image")
    if output_file_path:
        cv2.imwrite(output_file_path, imageEncrypted)

    # Close all windows
    cv2.destroyAllWindows()


# Function to decrypt the image
def decrypt_image():
    # Open file dialog to select encrypted image
    input_file_path = filedialog.askopenfilename(title="Select Encrypted Image")
    if not input_file_path:
        return

    # Retrieve the decryption key from the entry field
    decryption_key = decryption_key_entry.get()

    # Load encrypted image
    imageEncrypted = cv2.imread(input_file_path)
    rowEncrypted, columnOrig, depthOrig = imageEncrypted.shape

    # Convert encrypted image data to bytes
    encryptedBytes = imageEncrypted.tobytes()

    # Generate random salt
    salt = os.urandom(16)

    # Generate key from password using PBKDF2
    password = decryption_key.encode()  # Convert the key to bytes
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
    )
    key = kdf.derive(password)

    # Generate random nonce
    nonce = os.urandom(12)

    # Extract the authentication tag from the encrypted image
    tag = encryptedBytes[-16:]
    # tag=input("enter tag:")
    print(tag)

    ciphertext = encryptedBytes[:-16]

    # Decrypt
    cipher = Cipher(algorithms.AES(key), modes.GCM(nonce, tag), backend=default_backend())
    decryptor = cipher.decryptor()
    decryptedData = decryptor.update(ciphertext) + decryptor.finalize()

    # Remove padding
    block_size = algorithms.AES.block_size // 8
    padding_length = decryptedData[-1]
    decryptedDataUnpadded = decryptedData[:-padding_length] if padding_length < block_size else decryptedData

    # Convert bytes to decrypted image data
    decryptedImage = np.frombuffer(decryptedDataUnpadded, dtype=np.uint8).reshape(rowEncrypted, columnOrig,
                                                                                   depthOrig)

    # Display decrypted image
    cv2.imshow("Decrypted Image", decryptedImage)
    cv2.waitKey()

    # Save the decrypted image
    output_file_path = filedialog.asksaveasfilename(title="Save Decrypted Image")
    if output_file_path:
        cv2.imwrite(output_file_path, decryptedImage)

    # Close all windows
    cv2.destroyAllWindows()


# Create the main window
root = tk.Tk()
root.title("Image Encryption/Decryption")

# Open image
open_image_button = tk.Button(root, text="Open Image", command=open_image)
open_image_button.pack(pady=10)
input_image_label = tk.Label(root)
input_image_label.pack(pady=10)

# Create entry fields for encryption and decryption keys
encryption_key_entry_label = tk.Label(root, text="Encryption Key:")
encryption_key_entry_label.pack()
encryption_key_entry = tk.Entry(root, show="*")
encryption_key_entry.pack(pady=5)

# Create buttons for encryption and decryption
encrypt_button = tk.Button(root, text="Encrypt Image", command=encrypt_image, state=tk.DISABLED)
encrypt_button.pack(pady=10)

decrypt_button = tk.Button(root, text="Decrypt Image", command=decrypt_image)
decrypt_button.pack(pady=10)

decryption_key_entry_label = tk.Label(root, text="Decryption Key:")
decryption_key_entry_label.pack()
decryption_key_entry = tk.Entry(root, show="*")
decryption_key_entry.pack(pady=5)

# Run the GUI main loop
root.mainloop()


b'&2r\x89)\xa6\nr\xe7+\x82\xf7"\xf1\xa9\xf8'
b'\x93\x9dr\x81P8\x14cS%jWPzdp'


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\DEEPIKA\anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-a4606bbf81d1>", line 127, in decrypt_image
    decryptedData = decryptor.update(ciphertext) + decryptor.finalize()
  File "C:\Users\DEEPIKA\anaconda3\lib\site-packages\cryptography\hazmat\primitives\ciphers\base.py", line 229, in finalize
    data = self._ctx.finalize()
  File "C:\Users\DEEPIKA\anaconda3\lib\site-packages\cryptography\hazmat\backends\openssl\ciphers.py", line 200, in finalize
    raise InvalidTag
cryptography.exceptions.InvalidTag
